<div style="text-align:center">
    <h1>
        VSL RL : SUMO Simulation (DQL)
    </h1>
</div>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import sumo_env as env
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
import gym
import datetime

# Load the TensorBoard notebook extension
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard


/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Initialize the environment

In [2]:
import gym
from gym.envs.registration import register
from SUMOInitializeEnv import SUMOEnv_Initializer
register(
    id='SumoGUI-v0',
    entry_point='SUMOInitializeEnv:SUMOEnv_Initializer'
)
env = gym.make('SumoGUI-v0')
np.random.seed(0)
env.seed(0)
tf.__version__

'2.8.0'

Build model

In [3]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(env.action_space.size))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 16)                48        
                                                                 
 activation (Activation)     (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 102       
                                                                 
 activation_1 (Activation)   (None, 6)                 0         
                                                                 
Total params: 150
Trainable params: 150
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
%reload_ext tensorboard
log_dir = "/users/asmae/Desktop/PHD/rl_vsl_sumo/rl_vsl_sumo/gym_sumo/envs/logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = [TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

#### Deep Q Learning

In [15]:
import epsilon_greedy as EpsilonGreedy
#tf.debugging.experimental.enable_dump_debug_info("/users/asmae/Desktop/PHD/TS_LOGDIR", tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=50000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=env.action_space.size, memory=memory, nb_steps_warmup=10,
target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/users/asmae/Desktop/PHD/TS_LOGDIR", histogram_freq=1)
# Okay, now it's time to learn something! We visualize the training here for show, but this slows down training quite a lot. 
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
dqn.fit(env, nb_steps=2000, visualize=True, verbose=2 
          #,callbacks=tensorboard_callback
          )
env.close()
dqn.test(env, nb_episodes=1, nb_max_episode_steps= 2000, visualize=True)
env.close()

/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-03-05 12:52:13.553634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:52:13.565982: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:52:13.591829: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Training for 2000 steps ...
 Retrying in 1 seconds


Fontconfig warning: ignoring UTF-8: not a valid region tag
/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-03-05 12:52:14.773087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:52:14.787035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:52:14.793260: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


speed chosen = 19
State : [2.0, 0.0028441410693970416]
reward: 13.023475705878809
speed chosen = 19
State : [2.0, 0.0028441410693970416]
reward: 14.195379343320385
speed chosen = 19
State : [2.0, 0.0028441410693970416]
reward: 14.990758885542286
speed chosen = 19
State : [3.0, 0.004266211604095562]
reward: 16.2073208937073
speed chosen = 19
State : [3.0, 0.004266211604095562]
reward: 17.268560301834643
speed chosen = 19
State : [4.0, 0.005688282138794083]
reward: 17.56075777205967
speed chosen = 19
State : [4.0, 0.005688282138794083]
reward: 18.220435752277055
speed chosen = 19
State : [4.0, 0.005688282138794083]
reward: 19.087332406267315
speed chosen = 19
State : [5.0, 0.007110352673492606]
reward: 16.31848446077845
speed chosen = 19
State : [5.0, 0.007110352673492606]
reward: 17.007504682128236
speed chosen = 19
State : [5.0, 0.007110352673492606]
reward: 17.734328151267867
speed chosen = 19
State : [6.0, 0.008532423208191125]
reward: 17.84791156043796


/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
2022-03-05 12:52:15.303724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:52:15.380672: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:52:15.394179: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:52:15.402787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:52:15.537956: I tensorflow/c

speed chosen = 33
State : [7.0, 0.009954493742889647]
reward: 18.02720117368696
speed chosen = 19
State : [7.0, 0.009954493742889647]
reward: 18.886673401580133
speed chosen = 19
State : [7.0, 0.009954493742889647]
reward: 19.71752623582852
speed chosen = 19
State : [8.0, 0.011376564277588166]
reward: 19.309323328858575
speed chosen = 19
State : [7.7407407407407405, 2.564134025409189]
reward: 19.95613700733955
speed chosen = 19
State : [9.0, 0.012798634812286692]
reward: 19.133248937182998
speed chosen = 19
State : [8.407407407407407, 3.787641145541747]
reward: 19.95606680511614
speed chosen = 19
State : [9.0, 0.012798634812286692]
reward: 20.787703527054795
speed chosen = 19
State : [9.333333333333334, 2.442885587555736]
reward: 20.29960690393623
speed chosen = 19
State : [8.666666666666666, 3.9223199112843985]
reward: 20.937157307115136
speed chosen = 19
State : [11.0, 0.01564277588168373]
reward: 19.921666069795297
speed chosen = 19
State : [9.88888888888889, 7.421891459149707]
rewa

Fontconfig warning: ignoring UTF-8: not a valid region tag


speed chosen = 25
State : [2.0, 0.0028441410693970416]
reward: 7.9395253686234355
speed chosen = 19
State : [2.0, 0.0028441410693970416]
reward: 8.836158279466327
speed chosen = 19
State : [2.0, 0.0028441410693970416]
reward: 10.030070336047602
speed chosen = 19
State : [2.0, 0.0028441410693970416]
reward: 11.017301154859409
speed chosen = 19
State : [2.0, 0.0028441410693970416]
reward: 12.215725658241238
speed chosen = 19
State : [3.0, 0.004266211604095562]
reward: 13.839387053865037
speed chosen = 19
State : [4.0, 0.005688282138794083]
reward: 11.195545392580803
speed chosen = 19
State : [4.0, 0.005688282138794083]
reward: 12.149466419462824
speed chosen = 19
State : [4.0, 0.005688282138794083]
reward: 13.066903226878297
speed chosen = 19
State : [5.0, 0.007110352673492605]
reward: 13.509819775580274
speed chosen = 19
State : [6.0, 0.008532423208191125]
reward: 15.591271507705352
speed chosen = 19
State : [6.0, 0.008532423208191125]
reward: 16.522817361112555
speed chosen = 19
State 

In [11]:
%tensorboard --logdir={log_folder}